In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


In [23]:
# 首先就是创建我们的数据
# 采用里函数上的例子
lambda_ = 1
d = {"S":1, "M":2, "L":3}
x = np.array([[1, d['S']], [1, d['M']], [1, d['M']],
             [1, d['S']], [1, d['S']], [2, d['S']],
             [2, d['M']], [2, d['M']], [2, d['L']],
             [2, d['L']], [3, d['L']], [3, d['M']],
             [3, d['M']], [3, d['L']], [3, d['L']]])

y = np.array([-1, -1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1])

In [24]:
x

array([[1, 1],
       [1, 2],
       [1, 2],
       [1, 1],
       [1, 1],
       [2, 1],
       [2, 2],
       [2, 2],
       [2, 3],
       [2, 3],
       [3, 3],
       [3, 2],
       [3, 2],
       [3, 3],
       [3, 3]])

In [25]:
y

array([-1, -1,  1,  1, -1, -1, -1,  1,  1,  1,  1,  1,  1,  1, -1])

$P_\lambda(Y=1)=(9+\lambda_)/(15 + 2*\lambda_) = (9+0.2)/(15+2*0.2)=0.5974025974025974$
$P_\lambda(Y=-1)=(6+\lambda_)/(15 + 2*\lambda_) = (6+0.2)/(15+2*0.2)=0.40259740259740264$  
$P(X^{(1)}=1|Y=1) = (2+0.2)/(9+3*0.2)=0.22916666666666669 $  
$P(X^{(1)}=2|Y=1) = (3+0.2)/(9+3*0.2)=0.33333333333333337 $  
$P(X^{(1)}=3|Y=1) = (4+0.2)/(9+3*0.2)=0.43750000000000006 $  
$P(X^{(2)}=S|Y=1) = (1+0.2)/(9+3*0.2)=0.125 $   
$P(X^{(2)}=M|Y=1) = (4+0.2)/(9+3*0.2)=0.43750000000000006 $     
$P(X^{(2)}=L|Y=1) = (4+0.2)/(9+3*0.2)=0.43750000000000006 $  
$P(X^{(1)}=1|Y=-1) = (3+0.2)/(6+3*0.2)=0.4848484848484849 $  
$P(X^{(1)}=2|Y=-1) = (2+0.2)/(6+3*0.2)=0.33333333333333337 $   
$P(X^{(1)}=3|Y=-1) = (1+0.2)/(6+3*0.2)=0.18181818181818182 $   
$P(X^{(2)}=S|Y=-1) = (3+0.2)/(6+3*0.2)=0.4848484848484849 $  
$P(X^{(2)}=M|Y=-1) = (2+0.2)/(6+3*0.2)=0.33333333333333337 $   
$P(X^{(2)}=L|Y=-1) = (1+0.2)/(6+3*0.2)=0.18181818181818182 $   
so  
$P(Y=1)P(X^{(1)}=2|Y=1)P(X^{(2)}=S|Y=1) =0.5974025974025974* 0.33333333333333337*0.125=0.024891774891774892$  
$P(Y=-1)P(X^{(1)}=2|Y=-1)P(X^{(2)}=S|Y=-1) =0.40259740259740264* 0.33333333333333337*0.4848484848484849=0.06506624688442873$  

so, it should be -1.

In [26]:
data = np.hstack((x,y.reshape(len(y),1)))
data
# 只有一个参数!！！！

array([[ 1,  1, -1],
       [ 1,  2, -1],
       [ 1,  2,  1],
       [ 1,  1,  1],
       [ 1,  1, -1],
       [ 2,  1, -1],
       [ 2,  2, -1],
       [ 2,  2,  1],
       [ 2,  3,  1],
       [ 2,  3,  1],
       [ 3,  3,  1],
       [ 3,  2,  1],
       [ 3,  2,  1],
       [ 3,  3,  1],
       [ 3,  3, -1]])

In [42]:
class mynaivebeyes:
    def __init__(self,lambda_ = 0.2):
        self.lambda_ = lambda_
    
    def fit(self,x,y):
        py = {}
        # 首先我们先算我们的各个标签的对应的概率值
        I_y = {}
        for y_value in set(y):
            I_y[y_value] = len(np.where(y == y_value)[0])
            # 我靠牛逼演我是吧 返回值是一个元组里面夹一个np数组，还有一个括号里面是元素类型！！！！
            py[y_value] = (I_y[y_value] + lambda_)/(len(y) + len(np.unique(y))*lambda_)
        # 然后我们现在拿到我们的在标签等于不同值的情况下我们x在各个维度的分量等于不同特定值的概率
#         Ixy_same_time = {}
        py_then_x = {}
#         data = np.vstack(x,y.reshape(len(y),1))
        # 真拿捏了哥们 这不比参考文件的简单
        for i in range(x.shape[1]):
            for x_value in np.unique(x[:,i]):
                x_idx = np.where(x[:,i] == x_value)
                for y1 in set(y[x_idx]):
                    py_then_x["x{}={}|y={}".format(i+1,x_value,y1)] = (sum(y[x_idx]==y1) + lambda_)/ (I_y[y1] + len(np.unique(x[:,i])) * lambda_)
        
        self.py = py
        self.py_then_x = py_then_x
        self.I_y = I_y
        return I_y,py_then_x,py
    
    def predict(self,x_to_test):
        max_p = 0
        p = 1
        max_y = y[0]
        p1 = {}
        for y_value1 in set(y):
            p1[y_value1] =1 
            for i in range(x_to_test.shape[0]):
                # 这个if语句就是防止我们有一个新值 要加新键 同时下面还要防止有出现0的情况
                if "x{}={}|y={}".format(i+1,x_to_test[i],y_value1) not in py_then_x:
                    py_then_x["x{}={}|y={}".format(i+1,x_to_test[i],y_value1)] = lambda_/(I_y[y_value1]+len(np.unique(x[:,i]))*lambda_)
                p1[y_value1] =p1[y_value1]* py_then_x["x{}={}|y={}".format(i+1,x_to_test[i],y_value1)]
        # 下面还要搞一个for循环  
        for y_value2 in set(y):
            p1[y_value2] = p1[y_value2] * py[y_value2]
            if p1[y_value2] > max_p:
                max_p = p1[y_value2]
                max_y = y_value2
        return max_y,max_p,p1,py_then_x

In [43]:
model =  mynaivebeyes(lambda_ = 0.20)
model.fit(x,y)
model.predict(np.array([2, 1]))

(-1,
 0.06506624688442872,
 {1: 0.024891774891774892, -1: 0.06506624688442872},
 {'x1=1|y=1': 0.22916666666666669,
  'x1=1|y=-1': 0.4848484848484849,
  'x1=2|y=1': 0.33333333333333337,
  'x1=2|y=-1': 0.33333333333333337,
  'x1=3|y=1': 0.43750000000000006,
  'x1=3|y=-1': 0.18181818181818182,
  'x2=1|y=1': 0.125,
  'x2=1|y=-1': 0.4848484848484849,
  'x2=2|y=1': 0.43750000000000006,
  'x2=2|y=-1': 0.33333333333333337,
  'x2=3|y=1': 0.43750000000000006,
  'x2=3|y=-1': 0.18181818181818182,
  'x2=0|y=1': 0.020833333333333336,
  'x2=0|y=-1': 0.030303030303030307})

In [44]:
model =  mynaivebeyes(lambda_ = 0.20)
model.fit(x,y)
model.predict(np.array([2, 0]))

(1,
 0.004148629148629149,
 {1: 0.004148629148629149, -1: 0.004066640430276795},
 {'x1=1|y=1': 0.22916666666666669,
  'x1=1|y=-1': 0.4848484848484849,
  'x1=2|y=1': 0.33333333333333337,
  'x1=2|y=-1': 0.33333333333333337,
  'x1=3|y=1': 0.43750000000000006,
  'x1=3|y=-1': 0.18181818181818182,
  'x2=1|y=1': 0.125,
  'x2=1|y=-1': 0.4848484848484849,
  'x2=2|y=1': 0.43750000000000006,
  'x2=2|y=-1': 0.33333333333333337,
  'x2=3|y=1': 0.43750000000000006,
  'x2=3|y=-1': 0.18181818181818182,
  'x2=0|y=1': 0.020833333333333336,
  'x2=0|y=-1': 0.030303030303030307})